# Use camera and install lib

In [ ]:
##read image
import cv2
image_file='xxx.jpg'
image = cv2.imread(image_file)
##show image
cv2.imshow("window name",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
##get real-time image from camera
# Define a video capture object
#cv2.VideoCapture(0):own computer's webcamera
cap = cv2.VideoCapture(1) #Resolution (640,480):camera provided in class
while True:
    ret, frame = cap.read() #returns ret and the frame
    cv2.imshow("window name",frame)
    
    #press q to end the while loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
          break
            
cv2.destroyAllWindows()

In [ ]:
##install library for aruco marker
pip install opencv-contrib-python --user

# Construct global map

In [ ]:
import math

class Map:
    #compute_globalmap(self,image,start_id=0,goal_id=1):
    #  Generate globalmap 0:free 1:occupied 2:start 3:goal
    #  return self.globalmap
    #compute_pixel_to_distance(self):
    #  compute real world distance between two neighborhood pixels;unit:mm
    #  **need to get goal first then compute pixel_to_distance
    #  return self.pixel_to_distance
    #compute_start(self,image, start_id=0):
    #  Detect start by aruco marker
    #  Return start position in (x,y) coordinate; if not detected return None
    #  return self.start
    #compute_goal(self,image, goal_id=1):
    #  Detect goal by aruco marker
    #  Return goal position in (x,y) coordinate; if not detected return None
    #  return self.goal
    #get_pixel_to_distance(self):
    #  return self.pixel_to_distance
    #get_globalmap(self):
    #  return self.globalmap
    #get_start(self):
    #  return self.start
    #get_goal(self):
    #  return self.goal
    def compute_globalmap(self,image,start_id=0,goal_id=1):
        #Generate globalmap 0:free 1:occupied 2:start 3:goal
        #height, width = image.shape[:2]

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #Simple threshold method
        #ret,thresh = cv2.threshold(img,127,255,cv2.THRESH_BINARY)

        # Otsu's thresholding after Gaussian filtering
        blur = cv2.GaussianBlur(gray,(5,5),0)
        ret,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        #Generate Global map
        #from 0,255 convert to 0(free),1(occupied);change to (x,y) coordinate
        globalmap=np.transpose(np.array(thresh<255,dtype=int))

        #Detect start and goal by aruco marker
        #Return start and goal position in (x,y) coordinate; if not detected return None
        #start_id: start aruco marker id
        #goal_id=: goal aruco marker id

        #Define aruco marker detector
        arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
        arucoParams = cv2.aruco.DetectorParameters_create()

        #Detect aruco marker
        (corners, ids, rejected) = cv2.aruco.detectMarkers(image, arucoDict,parameters=arucoParams)
        # print(corners, ids, rejected)
        start=None
        goal=None
        s=2
        if ids is not None:
            for corner,number in zip(corners,ids):
                (topLeft, topRight, bottomRight, bottomLeft) = corner[0].astype(int)
                #print(topLeft, topRight, bottomRight, bottomLeft)
                centerpoint=(int((topLeft[0]+bottomRight[0])/2),int((topLeft[1]+bottomRight[1])/2))
                if number==start_id:
                    start=centerpoint
                    #print("start:",start)
                    globalmap[bottomRight[0]-s:topLeft[0]+s,topLeft[1]-s:bottomRight[1]+s]=2
                    #print(globlemap[bottomRight[0]:topLeft[0],topLeft[1]:bottomRight[1]])

                elif number==goal_id:
                    goal=centerpoint
                    #print("goal:",goal)
                    globalmap[bottomRight[0]-s:topLeft[0]+s,topLeft[1]-s:bottomRight[1]+s]=3
                    #print(globlemap[bottomRight[0]:topLeft[0],topLeft[1]:bottomRight[1]])
                    self.goal_topLeft=topLeft
                    self.goal_topRight= topRight

                else:
                    pass
        else: pass
        if start is None: print("NOT detect start point") 
        else:pass
        if goal is None: print("NOT detect goal point") 
        else:pass
        
        self.globalmap=globalmap
        self.start=start
        self.goal=goal

        return self.globalmap
    
    def compute_pixel_to_distance(self):
        #Need to run "compute_globalmap()" first
        #aruco marker:50mmx50mm
        #pixel_to_distance:return real world distance between two neighborhood pixels;unit:mm
        if self.goal is not None:
            pixel_to_distance=50/math.dist(self.goal_topLeft,self.goal_topRight) #unit:mm
            print("Distance between two pixel is:",pixel_to_distance,"mm")
        else:
            pixel_to_distance=None
            print("NOT detect goal point.Need to detect goal point first, then compute pixel_to_distance.")
        self.pixel_to_distance=pixel_to_distance
        return self.pixel_to_distance
    
    def get_pixel_to_distance(self):
        if self.pixel_to_distance is None:print("No value for pixel_to_distance")
        else:pass
        return self.pixel_to_distance
    
    def get_globalmap(self):
        return self.globalmap
    
    def get_start(self):
        #get the center point of start point
        if self.start is None: print("NOT detect start point") 
        else:pass
        return self.start
    
    def get_goal(self):
        #get the center point of goal point
        if self.goal is None: print("NOT detect goal point") 
        else:pass
        return self.goal

    def compute_start(self,image, start_id=0):
        #Detect start by aruco marker
        #Return start position in (x,y) coordinate; if not detected return None
        #start_id: start aruco marker id

        #Define aruco marker detector
        arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
        arucoParams = cv2.aruco.DetectorParameters_create()

        #Detect aruco marker
        (corners, ids, rejected) = cv2.aruco.detectMarkers(image, arucoDict,parameters=arucoParams)
        # print(corners, ids, rejected)
        start=None
        if ids is not None:
            for corner,number in zip(corners,ids):
                (topLeft, topRight, bottomRight, bottomLeft) = corner[0].astype(int)
                #print(topLeft, topRight, bottomRight, bottomLeft)
                centerpoint=(int((topLeft[0]+bottomRight[0])/2),int((topLeft[1]+bottomRight[1])/2))
                if number==start_id:
                    start=centerpoint
                    break
                else:
                    pass
        else: pass
        if start is None: print("NOT detect start point") 
        else:print("start position is:",start)
        self.start=start

        return self.start

    def compute_goal(self,image, goal_id=1):
        #Detect goal by aruco marker
        #Return goal position in (x,y) coordinate; if not detected return None
        #goal_id=: goal aruco marker id

        #Define aruco marker detector
        arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
        arucoParams = cv2.aruco.DetectorParameters_create()

        #Detect aruco marker
        (corners, ids, rejected) = cv2.aruco.detectMarkers(image, arucoDict,parameters=arucoParams)
        # print(corners, ids, rejected)
        goal=None
        if ids is not None:
            for corner,number in zip(corners,ids):
                (topLeft, topRight, bottomRight, bottomLeft) = corner[0].astype(int)
                #print(topLeft, topRight, bottomRight, bottomLeft)
                centerpoint=(int((topLeft[0]+bottomRight[0])/2),int((topLeft[1]+bottomRight[1])/2))
                if number==goal_id:
                    goal=centerpoint
                    self.goal_topLeft=topLeft
                    self.goal_topRight= topRight
                    break
                else:
                    pass
        else: pass
        if goal is None: print("NOT detect goal point") 
        else:print("goal position is:",goal)
        self.goal=goal
        
        return self.goal

# Get robot position and angle

In [ ]:
def get_robot_position(image,robot_id=3):
    #Detect robot position by aruco marker
    #Return robot position; if not detected return None
    #robot_id: robot aruco marker id

    #Define aruco marker detector
    arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
    arucoParams = cv2.aruco.DetectorParameters_create()

    robot=None

    #Detect Aruco marker
    (corners, ids, rejected) = cv2.aruco.detectMarkers(image, arucoDict,parameters=arucoParams)
    # print(corners, ids, rejected)
    if ids is not None:
        #print("aruco marker detected:",ids)
        for corner,number in zip(corners,ids):
            (topLeft, topRight, bottomRight, bottomLeft) = corner[0]
            centerpoint=(int((topLeft[0]+bottomRight[0])/2),int((topLeft[1]+bottomRight[1])/2))
            if number==robot_id:
                robot_position=centerpoint
                angle=angle_calculate(topLeft,topRight)
                break
            else:pass
    else:pass
    
    return robot_position,angle

def angle_calculate(pt1,pt2):  # function which returns angle between two points in the range of 0-359
    angle_list = list(range(0,360,1))
    x=pt2[0]-pt1[0] 
    y=pt2[1]-pt1[1]
    angle=int(math.degrees(math.atan2(y,x))) #takes 2 points nad give angle with respect to horizontal axis in range(-180,180)
    angle = angle_list[angle]
    
    return int(angle)

# Find vertex

In [ ]:
def compute_vertex(image,globalmap):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    globalmap=np.transpose(globalmap)
    #get rid of start and goal area
    gray[globalmap==2]=255
    gray[globalmap==3]=255

    # apply canny edge detection
    edges = cv2.Canny(gray, 60, 160)

    # apply morphology close
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    morph = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    # get contours 
    contours = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # get vertices
    vertices=[]
    for i in range(len(contours[0])):
        peri = cv2.arcLength(contours[0][i], True)
        approx = cv2.approxPolyDP(contours[0][i], 0.01 * peri, True)
        vertice=[]
        for i in range(len(approx)):
            vertice.append(approx[i][0].tolist())
        vertices.append(np.array(vertice))
        
    return vertices